<a href="https://colab.research.google.com/github/yexeh/class2022Spring/blob/main/API_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
import gradio as gr
import requests
import pandas as pd

In [ ]:
URL = ('http://apis.data.go.kr/B553766/smt-schedule/'
       'schedule?'
       'serviceKey=sBDDbJv7DKtHNT%2FYmiqZAGLgobklu9HF4w%2BJHiUVxEVEunMXCIl6WWbnip3q76BaIHBs20HMHouZgdoEmkdKFw%3D%3D'
       '&numOfRows=500'
       '&pageNo=1'
       '&stationCode=2640'
       '&week=DAY'
       '&trainDirection=UP'
       )
response = requests.get(URL)

In [ ]:
r = response.json()

In [ ]:
data = r['data']

station_name = [item['station_name'] for item in data]
arrive_time = [item['train_arrtime'] for item in data]
destination = [item['train_destination'] for item in data]

station_name = pd.Series(station_name)
arrive_time = pd.to_numeric(arrive_time)
destination = pd.Series(destination)

df = pd.DataFrame({ 'station_name':station_name, 'arrive_time':arrive_time, 'destination':destination })

df2 = df[ (df.arrive_time > 120000) & (df.arrive_time < 130000) ]


In [ ]:
df2

In [ ]:
def train_time(week, traindirection, time) :

  URL = ('http://apis.data.go.kr/B553766/smt-schedule/'
         'schedule?'
         'serviceKey=sBDDbJv7DKtHNT%2FYmiqZAGLgobklu9HF4w%2BJHiUVxEVEunMXCIl6WWbnip3q76BaIHBs20HMHouZgdoEmkdKFw%3D%3D'
         '&numOfRows=500'
         '&pageNo=1'
         '&stationCode=2640'
         '&week=' + week +
         '&trainDirection=' + traindirection 
         )
  
  response = requests.get(URL)
  r = response.json()

  data = r['data']

  station_name = [item['station_name'] for item in data]
  arrive_time = [item['train_arrtime'] for item in data]
  destination = [item['train_destination'] for item in data]

  station_name = pd.Series(station_name)
  arrive_time = pd.to_numeric(arrive_time)
  destination = pd.Series(destination)

  df = pd.DataFrame({ 'station_name':station_name, 'arrive_time':arrive_time, 'destination':destination })

  df2 = df[ ( df.arrive_time >= time*100 ) & ( df.arrive_time < time*100+10000 ) ]

  return df2

In [ ]:
iface = gr.Interface(train_time, [gr.inputs.Dropdown(['DAY', 'SAT', 'HOL']), gr.inputs.Dropdown(['UP', 'DOWN']), gr.inputs.Number(label='time')], outputs = ['dataframe'])
iface.launch(debug = True)